In [47]:
import pandas as pd
import ast
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np


df = pd.read_csv('movies_dataset_modificado.csv', low_memory=False)


In [20]:
type(df.at[2,"name_Genres"])

numpy.float64

In [46]:
df["name_Genres"] = df["name_Genres"].astype(str)
df["name_Genres"]


0         ['Animation', 'Comedy', 'Family']
1        ['Adventure', 'Fantasy', 'Family']
2                     ['Romance', 'Comedy']
3            ['Comedy', 'Drama', 'Romance']
4                                ['Comedy']
                        ...                
45461                   ['Drama', 'Family']
45462                             ['Drama']
45463       ['Action', 'Drama', 'Thriller']
45464                                    []
45465                                    []
Name: name_Genres, Length: 45466, dtype: object

In [48]:
df["name_Genres"] = df["name_Genres"].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").replace(", ", ","))
df["name_Genres"]

0         Animation,Comedy,Family
1        Adventure,Fantasy,Family
2                  Romance,Comedy
3            Comedy,Drama,Romance
4                          Comedy
                   ...           
45461                Drama,Family
45462                       Drama
45463       Action,Drama,Thriller
45464                            
45465                            
Name: name_Genres, Length: 45466, dtype: object

In [124]:
data = df[['title', 'vote_average', 'name_Genres', "release_year", "name_production_companies","overview"]]

In [164]:
type(df.at[4,"release_year"])

numpy.float64

In [165]:
#prueba utilizada
df1 = df.head(30000).reset_index(drop=True)
titles = df1['title']
titles

0                                Toy Story
1                                  Jumanji
2                         Grumpier Old Men
3                        Waiting to Exhale
4              Father of the Bride Part II
                       ...                
29995                      Seventeen Again
29996                        Sweet Sixteen
29997    The Disappearance of Garcia Lorca
29998              The Dramatics: A Comedy
29999                         Up the Creek
Name: title, Length: 30000, dtype: object

In [166]:
#Primer intento de ML, equivocado porque esto no se considera Machine Lerning aunque devuelva una lista
similarity_matrix =  pairwise_distances(data.pivot_table(index='name_Genres', columns=['release_year']).fillna(0), metric="cosine")

C:\Users\tonto\AppData\Local\Temp\ipykernel_16728\3122387956.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  similarity_matrix =  pairwise_distances(data.pivot_table(index='name_Genres', columns=['release_year']).fillna(0), metric="cosine")


In [167]:
similarity_matrix = pairwise_distances(data.pivot_table(index='name_Genres', columns='release_year', values='vote_average', fill_value=0), metric='cosine')


In [168]:
def recomendacion(titulo):
    
    
   
    
    # Buscar la película seleccionada por el usuario en el DataFrame
    idx = data[data['title'] == titulo].index[0]
    
    # Ordenar la matriz de similaridad para obtener las películas más similares
    similar_movie_indices = np.argsort(similarity_matrix[idx])
    
    # Seleccionar las primeras 5 películas más similares y obtener sus títulos
    recommended_movies = []
    for i in similar_movie_indices:
        if data.iloc[i]['title'] != titulo:
            recommended_movies.append(data.iloc[i]['title'])
        if len(recommended_movies) == 5:
            break
    
    # Devolver una lista con los títulos de las películas recomendadas
    return {'lista recomendada': recommended_movies}


In [169]:
recomendacion(("The Exorcist III"))


{'lista recomendada': ['The Yards',
  "Rosemary's Baby",
  'Best Men',
  'Lured',
  'Farinelli']}

In [171]:
pelicula_entrada = data.loc[data['title'] ==   "Rosemary's Baby"]
generos_entrada = pelicula_entrada['name_Genres'].values[0]

generos_entrada

'Horror,Drama,Mystery'

In [172]:
pelicula_entrada = data.loc[data['title'] ==   'Farinelli']
generos_entrada = pelicula_entrada['release_year'].values[0]

generos_entrada

1994.0

In [173]:
print(data['release_year'].isnull().sum())

90


In [765]:
#Primer intento real de ML
# Importar bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import NearestNeighbors
# Crear conjunto de características y variable objetivo
X = df[['name_Genres', 'release_year']]
y = df['title']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Procesamiento de características
mlb = MultiLabelBinarizer()
genres_one_hot = mlb.fit_transform(X_train['name_Genres'])
vectorizer = TfidfVectorizer()
release_year_tfidf = vectorizer.fit_transform(X_train['release_year'])

# Definir modelo de recomendación
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('genres', genres_one_hot),
        ('year', release_year_tfidf)
    ])),
    ('clf', NearestNeighbors(metric='cosine'))
])

# Entrenar modelo de recomendación
pipeline.fit(X_train, y_train)

# Predecir películas similares
def recomendacion(titulo):
    indices = pipeline.kneighbors(X_test.loc[X_test['title']==titulo])[1][0]
    return list(y_test.iloc[indices])


AttributeError: 'float' object has no attribute 'lower'

In [746]:
df = pd.read_csv('movies_dataset_modificado.csv', low_memory=False)
df["name_Genres"]
type(df.at[1,"name_Genres"])


str

Apartir de aqui comienza el resultado final

In [747]:

#Apartir de aqui comienza el resultado final
df = pd.read_pickle('movies_dataset_modificado.pickle')
df["name_Genres"]
type(df.at[1,"name_Genres"])

list

In [748]:

data = df[['title', 'name_Genres', 'vote_average']]
data

,title,name_Genres,vote_average
0,Toy Story,"[Animation, Comedy, Family]",7.7
1,Jumanji,"[Adventure, Fantasy, Family]",6.9
2,Grumpier Old Men,"[Romance, Comedy]",6.5
3,Waiting to Exhale,"[Comedy, Drama, Romance]",6.1
4,Father of the Bride Part II,[Comedy],5.7
...,...,...,...
45461,Subdue,"[Drama, Family]",4.0
45462,Century of Birthing,[Drama],9.0
45463,Betrayal,"[Action, Drama, Thriller]",3.8
45464,Satan Triumphant,[],0.0


In [749]:
data['vote_average'] = data['vote_average'].fillna(5.6)


data

C:\Users\tonto\AppData\Local\Temp\ipykernel_12108\2735702412.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['vote_average'] = data['vote_average'].fillna(5.6)


,title,name_Genres,vote_average
0,Toy Story,"[Animation, Comedy, Family]",7.7
1,Jumanji,"[Adventure, Fantasy, Family]",6.9
2,Grumpier Old Men,"[Romance, Comedy]",6.5
3,Waiting to Exhale,"[Comedy, Drama, Romance]",6.1
4,Father of the Bride Part II,[Comedy],5.7
...,...,...,...
45461,Subdue,"[Drama, Family]",4.0
45462,Century of Birthing,[Drama],9.0
45463,Betrayal,"[Action, Drama, Thriller]",3.8
45464,Satan Triumphant,[],0.0


In [750]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Preprocesamiento de datos variables categoricas codificadas

mlb = MultiLabelBinarizer()
data = data.join(pd.DataFrame(mlb.fit_transform(data.pop('name_Genres')),
                          columns=mlb.classes_, 
                          index=data.index))


In [661]:
data

,title,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,Crime,Documentary,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
0,Toy Story,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,Subdue,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45462,Century of Birthing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45463,Betrayal,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
45464,Satan Triumphant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [751]:
from sklearn.model_selection import train_test_split

# Separar los datos en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)


In [752]:
from sklearn.neighbors import NearestNeighbors

# Crear una matriz de similitud entre películas
model = NearestNeighbors(metric='hamming', algorithm='auto')
model.fit(train_data.drop(['title'], axis=1))

NearestNeighbors(metric='hamming')

In [753]:
import numpy as np

def get_recommendations( movie_title):
    # Obtener el índice de la película en el conjunto de datos
    idx = data[data['title'] == movie_title].index[0]
    # Obtener las películas más similares
    distances, indices = model.kneighbors(data.iloc[idx, 1:].values.reshape(1, -1), n_neighbors=6)
    # Obtener los títulos de las películas recomendadas
    recommended_movies = [data.iloc[indices[0][i+1]]['title'] for i in range(5)]
    return recommended_movies


In [754]:

# Ejemplo de recomendación
get_recommendations("Akira")


c:\Users\tonto\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['The Fern Flower',
 'Don 2: The King is Back',
 'Arena',
 'Popstar',
 'The Salt of the Earth']

In [759]:
pelicula_entrada = df.loc[df['title'] ==   'The Salt of the Earth']
generos_entrada = pelicula_entrada['name_Genres'].values[0]

generos_entrada

['Documentary']

In [590]:
train_data

,title,vote_average,release_year,Action,Adventure,Animation,Aniplex,BROSTA TV,Carousel Productions,Comedy,...,Romance,Science Fiction,Sentai Filmworks,TV Movie,Telescene Film Group Productions,The Cartel,Thriller,Vision View Entertainment,War,Western
33257,Triple Dare,5.3,2006.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13648,Euphoria,6.0,2006.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
14690,Dragon Hunters,6.5,2008.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28348,Hot Tub Time Machine 2,5.0,2015.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
18211,The 12 Dogs of Christmas,4.4,2005.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35483,Chrysalis,4.9,2014.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
39296,Phas Gaye Re Obama,6.2,2010.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2693,Universal Soldier,5.8,1992.0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
8076,Blue Spring,6.4,2001.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [764]:
data.to_pickle('Data_ML.pickle')

In [763]:
df

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,name_production_companies,id_production_companies,name_production_countries,iso_3166_1_production_countries,name_spoken_languages,iso_639_1_spoken_languages,release_year,release_month,release _day,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,[Pixar Animation Studios],[3],[United States of America],[US],[English],[en],1995.0,Octubre,Lunes,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"[TriStar Pictures, Teitler Film, Interscope Co...","[559, 2550, 10201]",[United States of America],[US],"[English, Français]","[en, fr]",1995.0,Diciembre,Viernes,4.043035
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"[Warner Bros., Lancaster Gate]","[6194, 19464]",[United States of America],[US],[English],[en],1995.0,Diciembre,Viernes,0.000000
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,[Twentieth Century Fox Film Corporation],[306],[United States of America],[US],[English],[en],1995.0,Diciembre,Viernes,5.090760
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,"[Sandollar Productions, Touchstone Pictures]","[5842, 9195]",[United States of America],[US],[English],[en],1995.0,Febrero,Viernes,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0.0,439050,fa,Rising and falling between a man and woman.,0.072051,NaT,0.0,90.0,Released,Rising and falling between a man and woman,...,[],[],[Iran],[IR],[فارسی],[fa],NaN,NaN,NaN,0.000000
45462,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,...,[Sine Olivia],[19653],[Philippines],[PH],[],[tl],2011.0,Noviembre,Jueves,0.000000
45463,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,[American World Pictures],[6165],[United States of America],[US],[English],[en],2003.0,Agosto,Viernes,0.000000
45464,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,...,[Yermoliev],[88753],[Russia],[RU],[],[],1917.0,Octubre,Domingo,0.000000
